In [11]:
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import pandas as pd

In [12]:
import numpy as np
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [13]:
# Unpickling here
dct = unpickle('ESIM_data/ByteDance/train_novelty_300.pickle')

In [14]:
keys = list(dct.keys())[:5]
print(keys)

['0', '3', '1', '2', '9']


In [15]:
# Checking sample
dct['0']

array([[ 0.9724187 ,  1.        , -0.6444988 , -0.99894625, -0.44577026,
        -0.5849954 , -0.99997544,  1.        ,  0.99968827,  0.9989656 ,
         0.9935129 , -0.97580147,  0.9990773 , -0.99999624,  0.10797489,
         0.990064  ,  0.49736542,  0.9997658 ,  0.88579935,  0.99999934,
         0.99199766,  0.9999997 ,  0.9992458 , -0.11192565,  0.9999928 ,
         0.9947987 ,  0.99871606,  0.9983769 ,  0.99668175, -0.99999994,
         0.7510062 ,  0.98782706, -0.36575687, -0.9728139 ,  1.        ,
        -0.999724  ,  0.9829801 , -0.89911366, -1.        , -0.9950432 ,
        -0.7986764 , -0.99978554, -0.99999475, -0.9494503 ,  0.17233   ,
         0.7843789 , -0.96700275,  0.9599521 , -0.96147346,  0.99842143,
         0.99961144,  0.99969035, -0.88511014, -0.99342364, -0.8722546 ,
        -0.97483176,  0.7789001 , -0.99996823,  0.9897508 ,  1.        ,
        -0.94398075,  1.        ,  1.        , -0.7013507 , -0.9996549 ,
        -0.99999994, -1.        ,  0.99998546,  0.7

In [16]:
# Examining its shape
dct['0'].shape

(1, 300)

In [17]:
# Stacking the arrays to create a single feature matrix
novelty_feature = np.stack(list(dct.values()))
novelty_feature.shape

(320552, 1, 300)

In [18]:
# Removing the middle dimension
novelty_feature.resize((320552, 300))
novelty_feature.shape

(320552, 300)

In [19]:
# Reading the test set
dct_test = unpickle('ESIM_data/ByteDance/test_novelty_300.pickle')
test_feature = np.stack(list(dct_test.values()))
test_feature.resize(list(test_feature.shape)[0], 300)
test_feature.shape

(80126, 300)

In [20]:
# TODO - Cateogorically encode the lables layer
import pandas as pd
from sklearn.preprocessing import LabelEncoder
train_df = pd.read_csv('../ByteDance_Dataset/train.csv')
print(train_df.columns)
le = LabelEncoder()
train_df['bd_label'] = le.fit_transform(train_df['bd_label'])
# train_df.label = train_df.label.astype('category').cat.codes #unrelated-2; disagreed-1; agreed-0
train_df.head()

Index(['id', 'tid1', 'tid2', 'title1_zh', 'title2_zh', 'title1_en',
       'title2_en', 'bd_label', 'Novelty_Labels', 'Emotion_1',
       'Emotion_1_comb', 'Quora_Labels', 'Quora_Labels_new'],
      dtype='object')


,id,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,bd_label,Novelty_Labels,Emotion_1,Emotion_1_comb,Quora_Labels,Quora_Labels_new
0,0,0,1,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",2,contradiction,sadness,anger,novel,novel
1,3,2,3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,2,neutral,anger,anger,novel,novel
2,1,2,4,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,"""If you do not come to Shenzhen, sooner or lat...",The GDP overtopped Hong Kong? Shenzhen clarifi...,2,neutral,anger,anger,novel,novel
3,2,2,5,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP topped Hong Kong last year? She...,2,contradiction,anger,anger,novel,novel
4,9,6,7,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,"""How to discriminate oil from gutter oil by me...",It took 30 years of cooking oil to know that o...,0,contradiction,sadness,sadness,novel,duplicate


In [21]:
# Loading the test set
test_df = pd.read_csv('../ByteDance_Dataset/test_merged.csv')
print(test_df.columns)
#test_df.Category = test_df.Category.astype('category').cat.codes
test_df['bd_label'] = le.transform(test_df['bd_label'])
test_df.head()

Index(['id', 'tid1', 'tid2', 'title1_zh', 'title2_zh', 'title1_en',
       'title2_en', 'bd_label', 'Weight', 'Usage', 'Novelty_Labels',
       'Emotion_1', 'Emotion_1_comb', 'Quora_Labels', 'Quora_Labels_new',
       'Emotion_1_bdtrain', 'Quora_Labels_new_1'],
      dtype='object')


,id,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,bd_label,Weight,Usage,Novelty_Labels,Emotion_1,Emotion_1_comb,Quora_Labels,Quora_Labels_new,Emotion_1_bdtrain,Quora_Labels_new_1
0,321187,167562,59521,萨拉赫人气爆棚!埃及总统大选未参选获百万选票 现任总统压力山大,辟谣！里昂官方否认费基尔加盟利物浦，难道是价格没谈拢？,egypt 's presidential election failed to win m...,Lyon! Lyon officials have denied that Felipe F...,2,0.0625,Private,neutral,anger,disgust,novel,novel,anger,novel
1,321190,167564,91315,萨达姆被捕后告诫美国的一句话，发人深思,10大最让美国人相信的荒诞谣言，如蜥蜴人掌控着美国,A message from Saddam Hussein after he was cap...,The Top 10 Americans believe that the Lizard M...,2,0.0625,Public,contradiction,anger,anger,novel,duplicate,anger,duplicate
2,321189,167563,167564,萨达姆此项计划没有此国破坏的话，美国还会对伊拉克发动战争吗,萨达姆被捕后告诫美国的一句话，发人深思,Will the United States wage war on Iraq withou...,A message from Saddam Hussein after he was cap...,2,0.0625,Private,neutral,anger,fear,novel,duplicate,sadness,duplicate
3,321193,167564,160994,萨达姆被捕后告诫美国的一句话，发人深思,被绞刑处死的萨达姆是替身？他的此男人举动击破替身谣言！,A message from Saddam Hussein after he was cap...,The hanging Saddam is a surrogate? This man's ...,2,0.0625,Public,contradiction,fear,anger,novel,duplicate,anger,duplicate
4,321191,167564,15084,萨达姆被捕后告诫美国的一句话，发人深思,中国川贝枇杷膏在美国受到热捧？纯属谣言！,A message from Saddam Hussein after he was cap...,Chinese loquat loquat plaster in America? Pure...,2,0.0625,Public,contradiction,joy,anger,novel,duplicate,sadness,novel


In [22]:
# Loading the pre-trained Glove embeddings
embeddings_dict = {}
with open("../resources/glove.6B.200d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [23]:
def find_closest_embeddings(embedding):
    return sorted(embeddings_dict.keys(), key=lambda word: spatial.distance.euclidean(embeddings_dict[word], embedding))

In [24]:
print(embeddings_dict["original"])
print(find_closest_embeddings(embeddings_dict["original"])[1:6])

[-0.45048   -0.1449     0.069873  -0.2611     0.51599    0.2649
 -0.37062   -0.4043    -0.047766  -0.85103    0.15076   -0.3983
 -0.30388    0.13779    0.31623   -0.27125    0.45444    0.93036
  0.37158   -0.13067    0.016844   2.2395     0.21558   -0.97138
 -0.15899    0.5531    -0.045112  -0.76692    0.0094216 -0.12936
 -0.21059   -0.11888    0.1508    -0.2525    -0.22782   -0.53595
 -0.40099   -0.58793    0.059262  -0.64623   -0.20917   -0.03534
 -0.034241  -0.08936   -0.16375    0.36763    0.82737   -0.10209
  0.19804    0.4031    -0.36257   -0.072119   0.2679    -0.20291
  0.10427    0.24153   -0.06382    0.4669    -0.12288    0.11546
 -0.11928    0.12932   -0.8338    -0.82749    0.014886  -0.57084
 -0.58857   -0.089826   0.39842    0.022715   0.54963   -0.30594
  0.058506   0.33867    0.17773    0.30477   -0.13231    0.1711
 -0.26207   -0.20595    0.58286   -0.090458   0.026476   0.090705
  0.20869   -0.22999   -0.51844   -0.82133    0.81661   -0.63622
  0.10171    1.0201     0.3

In [25]:
print(embeddings_dict["duplicate"])
print(find_closest_embeddings(embeddings_dict["duplicate"])[1:6])

[-0.11542    0.62583    0.0031159 -0.27893    0.67254   -0.45319
 -0.16397   -0.32293   -0.26415   -0.25232   -0.52077   -0.088397
 -0.69601   -0.16587    0.12641   -0.60435   -0.45325    0.34664
 -0.37659   -0.44404   -0.54856    0.46296    0.16677    0.53835
 -0.51188   -0.026023   0.64821   -0.6264     0.44719   -0.15889
 -0.2688     0.035679   0.29732   -0.048158  -0.0096131 -0.37165
 -0.67745   -0.5086     0.64688    0.1884     0.19655    0.034364
  0.29706    0.20052   -0.016906   0.0406     0.56526   -0.55097
 -0.10443   -0.22204   -0.03948   -0.90869   -0.14798    0.19678
 -0.15683    0.62182   -0.0029273 -0.4239    -0.063591  -0.12829
 -0.39695    0.21382   -0.10626    0.033134   0.5027    -0.35057
  0.070328  -0.28327    0.29084    0.33744    0.56324   -0.46098
  0.63119    0.21986   -0.73121    0.13872    0.0082016 -0.30085
 -0.25409    0.67545    0.36366    0.54803    0.68694    0.18578
  0.29969   -0.55681   -0.41297   -0.3755     0.3044    -0.43858
  0.36564    0.2197    

In [26]:
# Storing for further use
novel = embeddings_dict["original"]
duplicate = embeddings_dict["duplicate"]
print(novel.shape)

(200,)


In [27]:
# Reading Labels info
train_bias = []
test_bias = []
zero_vector = np.zeros((200,))
for i, row in train_df.iterrows():
    if row['Novelty_Labels'] == 'contradiction':
        train_bias.append(novel)
    elif row['Novelty_Labels'] == 'entailment':
        train_bias.append(duplicate)
    else:
        train_bias.append(zero_vector)
for i, row in test_df.iterrows():
    if row['Novelty_Labels'] == 'contradiction':
        test_bias.append(novel)
    elif row['Novelty_Labels'] == 'entailment':
        test_bias.append(duplicate)
    else:
        test_bias.append(zero_vector)
train_bias = np.stack(train_bias)
test_bias = np.stack(test_bias)
print('Train bias', train_bias.shape)
print('Test bias', test_bias.shape)

Train bias (320552, 200)
Test bias (80126, 200)


In [28]:
# Performing PCA to reduce the dimensions
from sklearn.decomposition import PCA
pca = PCA(n_components=200)
train_novelty_feature = pca.fit_transform(novelty_feature)
test_novelty_feature = pca.transform(test_feature)
print('Train shape is', train_novelty_feature.shape)
print('Test shape is', test_novelty_feature.shape)

Train shape is (320552, 200)
Test shape is (80126, 200)


In [29]:
bd_train_nt_feature = np.add(train_novelty_feature, train_bias)
bd_test_nt_feature = np.add(test_novelty_feature, test_bias)
print('Train', bd_train_nt_feature.shape)
print('Test', bd_test_nt_feature.shape)
print('Max Train value', np.amax(bd_train_nt_feature))
print('Min Train value', np.amin(bd_train_nt_feature))
print('Max Test value', np.amax(bd_test_nt_feature))
print('Min Test value', np.amin(bd_test_nt_feature))

Train (320552, 200)
Test (80126, 200)
Max Train value 14.991167068481445
Min Train value -12.164078265428543
Max Test value 15.394601114327088
Min Test value -12.056631594896317


In [30]:
# Applying simple logistic regression
from sklearn import linear_model
from sklearn import metrics

In [31]:
# Removing the unrelated samples from both train and test
#print(type(train_df['bd_label'] == 2))
result = np.where(train_df['bd_label'] == 2)
reduced_bd_nt_train = np.delete(bd_train_nt_feature, result[0], axis=0)
print('Train shape', reduced_bd_nt_train.shape)
reduced_train_labels = np.delete(train_df['bd_label'].values, result[0])
print('Train labels', reduced_train_labels)
result_test = np.where(test_df['bd_label']==2)
reduced_bd_nt_test = np.delete(bd_test_nt_feature, result_test[0], axis=0)
print('Test shape', reduced_bd_nt_test.shape)
reduced_test_labels = np.delete(test_df['bd_label'].values, result_test[0])
reduced_test_weights = np.delete(test_df['Weight'].values, result_test[0])
print('Test labels', reduced_test_labels)

Train shape (101239, 200)
Train labels [0 0 0 ... 1 0 0]
Test shape (28746, 200)
Test labels [0 0 0 ... 0 0 0]


In [35]:
# Considering the new emotion representations
train_emotion_dict_pre = unpickle('../Proposed_Model/ByteDance_Data/train_ag_dg_premise.tsv_k_bal_bin.pickle')
train_emotion_dict_hyp = unpickle('../Proposed_Model/ByteDance_Data/train_ag_dg_hyp.tsv_k_bal_bin.pickle')

test_emotion_dict_pre = unpickle('../Proposed_Model/ByteDance_Data/test_ag_dg_premise.tsv_k_bal_bin.pickle')
test_emotion_dict_hyp = unpickle('../Proposed_Model/ByteDance_Data/test_ag_dg_hyp.tsv_k_bal_bin.pickle')

train_em_pre = np.stack(list(train_emotion_dict_pre.values()))
train_em_hyp = np.stack(list(train_emotion_dict_hyp.values()))
test_em_pre  = np.stack(list(test_emotion_dict_pre.values()))
test_em_hyp  = np.stack(list(test_emotion_dict_hyp.values()))

print('Train premise', train_em_pre.shape)
print('Train hyp', train_em_hyp.shape)
print('Test premise', test_em_pre.shape)
print('Test hyp', test_em_hyp.shape)

Train premise (101239, 768)
Train hyp (101239, 768)
Test premise (28746, 768)
Test hyp (28746, 768)


In [36]:
# Adding the premise and hypothesis
train_em = np.add(train_em_pre, train_em_hyp)
test_em = np.add(test_em_pre, test_em_hyp)
print('Train', train_em.shape)
print('Test', test_em.shape)
print('Max Train value', np.amax(train_em))
print('Min Train value', np.amin(train_em))
print('Max Test value', np.amax(test_em))
print('Min Test value', np.amin(test_em))

Train (101239, 768)
Test (28746, 768)
Max Train value 1.9978583
Min Train value -1.9980206
Max Test value 1.9978583
Min Test value -1.9980199


In [37]:
# Performing PCA to reduce the dimensions
pca = PCA(n_components=200)
train_em = pca.fit_transform(train_em)
test_em = pca.transform(test_em)
print('Train shape is', train_em.shape)
print('Test shape is', test_em.shape)

Train shape is (101239, 200)
Test shape is (28746, 200)


In [38]:
emotion_true = embeddings_dict['anticipation']+embeddings_dict['sadness']+embeddings_dict['joy']+embeddings_dict['trust']
emotion_false = embeddings_dict['anger']+embeddings_dict['fear']+embeddings_dict['disgust']+embeddings_dict['surprise']
print('True emotion', emotion_true.shape)
print('Fake emotion', emotion_false.shape)

True emotion (200,)
Fake emotion (200,)


In [39]:
train_ag_dg = pd.read_csv('../Proposed_Model/ByteDance_Data/train_ag_dg_only.csv')
test_ag_dg = pd.read_csv('../Proposed_Model/ByteDance_Data/test_ag_dg_only.csv')
train_hy_df = pd.read_csv('../Proposed_Model/ByteDance_Data/train_ag_dg_hyp.tsv_k_bal_numb_predictions_bin.csv')
train_pre_df = pd.read_csv('../Proposed_Model/ByteDance_Data/train_ag_dg_premise.tsv_k_bal_numb_predictions_bin.csv')
test_hy_df = pd.read_csv('../Proposed_Model/ByteDance_Data/test_ag_dg_hyp.tsv_k_bal_numb_predictions_bin.csv')
test_pre_df = pd.read_csv('../Proposed_Model/ByteDance_Data/test_ag_dg_premise.tsv_k_bal_numb_predictions_bin.csv')
assert len(train_ag_dg) == len(train_hy_df) == len(train_pre_df)
assert len(test_ag_dg) == len(test_hy_df) == len(test_pre_df)

In [40]:
# New kind of adding scaffold labels
train_bias_em = []
test_bias_em = []
zero_vector = np.zeros((200,))
for i in range(len(train_ag_dg)):
    pre = train_pre_df.loc[i, 'Emotion_Label']
    hyp = train_hy_df.loc[i, 'Emotion_Label']
    if train_ag_dg.loc[i, 'bd_label'] == 'agreed' and (pre==1 and hyp==1):
        train_bias_em.append(emotion_false)
    elif train_ag_dg.loc[i, 'bd_label'] == 'disagreed' and (pre==1 and hyp==0):
        train_bias_em.append(emotion_true)
    else:
        train_bias_em.append(zero_vector)
for i in range(len(test_ag_dg)):
    pre = test_pre_df.loc[i, 'Emotion_Label']
    hyp = test_hy_df.loc[i, 'Emotion_Label']
    if test_ag_dg.loc[i, 'bd_label'] == 'agreed' and (pre==1 and hyp==1):
        test_bias_em.append(emotion_false)
    elif test_ag_dg.loc[i, 'bd_label'] == 'disagreed' and (pre==1 and hyp==0):
        test_bias_em.append(emotion_true)
    else:
        test_bias_em.append(zero_vector)
train_bias_em = np.stack(train_bias_em)
test_bias_em = np.stack(test_bias_em)
print('Train bias', train_bias_em.shape)
print('Test bias', test_bias_em.shape)

Train bias (101239, 200)
Test bias (28746, 200)


In [41]:
bd_train_et_feature = np.add(train_em, train_bias_em)
bd_test_et_feature = np.add(test_em, test_bias_em)
print('Train', bd_train_et_feature.shape)
print('Test', bd_test_et_feature.shape)
print('Max Train value', np.amax(bd_train_et_feature))
print('Min Train value', np.amin(bd_train_et_feature))
print('Max Test value', np.amax(bd_test_et_feature))
print('Min Test value', np.amin(bd_test_et_feature))

Train (101239, 200)
Test (28746, 200)
Max Train value 41.916015625
Min Train value -36.55155563354492
Max Test value 41.9017448425293
Min Test value -36.551055908203125


In [42]:
combined_bd_train = np.add(reduced_bd_nt_train, bd_train_et_feature)
combined_bd_test = np.add(reduced_bd_nt_test, bd_test_et_feature)
print('Combined Train', combined_bd_train.shape)
print('Combined Test', combined_bd_test.shape)

Combined Train (101239, 200)
Combined Test (28746, 200)


In [45]:
# Final Combined Logistic Regression Model
lg_reg_combine = linear_model.LogisticRegression(max_iter = 5000)
lg_reg_combine.fit(combined_bd_train, reduced_train_labels)

LogisticRegression(max_iter=5000)

In [46]:
y_pred_combine = lg_reg_combine.predict(combined_bd_test)
print("Accuracy of Combined Reduced Logistic Regression model is:",
#metrics.accuracy_score(test_df['Expected_bd'].values, y_pred, sample_weight = test_df['Weight'].values)*100)
metrics.accuracy_score(reduced_test_labels, y_pred_combine, sample_weight = reduced_test_weights)*100)
print(classification_report(reduced_test_labels, y_pred_combine, target_names = ['agreed', 'disagreed'], sample_weight = reduced_test_weights))

Accuracy of Combined Reduced Logistic Regression model is: 91.75392673579879
              precision    recall  f1-score   support

      agreed       0.91      1.00      0.95 1805.000009024593
   disagreed       0.99      0.48      0.64 334.1999999999896

    accuracy                           0.92 2139.200009024583
   macro avg       0.95      0.74      0.80 2139.200009024583
weighted avg       0.92      0.92      0.91 2139.200009024583

